In [1]:
# pip install countryinfo

import pandas as pd
import re
from countryinfo import CountryInfo

## Load in the Cases data

In [2]:
!curl -L 'https://www.who.int/entity/immunization/monitoring_surveillance/data/incidence_series.xls' -o '../raw_data/cases/incidence_series.xls'
# -L to tell cURL to follow the redirect


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   183  100   183    0     0    690      0 --:--:-- --:--:-- --:--:--   690
100  611k  100  611k    0     0   528k      0  0:00:01  0:00:01 --:--:-- 17.5M


In [3]:
casesDF = pd.read_excel(
    '../raw_data/cases/incidence_series.xls', 
    index_col=None, 
    sheet_name='Measles')

In [4]:
casesDF.head()

,WHO_REGION,ISO_code,Cname,Disease,2018,2017,2016,2015,2014,2013,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,EMR,AFG,Afghanistan,measles,2012.0,1511.0,638.0,1154.0,492.0,430.0,...,1170.0,4561.0,10357.0,8107.0,14457.0,16199.0,18808.0,20320.0,31107.0,32455.0
1,EUR,ALB,Albania,measles,1469.0,12.0,17.0,NaN,NaN,0.0,...,136034.0,0.0,0.0,0.0,0.0,0.0,17.0,3.0,NaN,NaN
2,AFR,DZA,Algeria,measles,3356.0,112.0,41.0,63.0,0.0,25.0,...,4169.0,2634.0,2500.0,3975.0,20114.0,22553.0,22126.0,29584.0,20849.0,15527.0
3,EUR,AND,Andorra,measles,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFR,AGO,Angola,measles,57.0,29.0,53.0,119.0,11699.0,8523.0,...,19820.0,21009.0,13368.0,15580.0,22822.0,22685.0,22589.0,30067.0,19714.0,29656.0


Remove and rename columns for consistency

In [5]:
# Also remove 0218 as we have new data for this column
casesDF.drop(['WHO_REGION', 'Disease', '2018'], axis=1, inplace=True)
casesDF.rename(index=str, columns={"ISO_code":"code","Cname":"name"}, inplace=True)
casesDF.head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,AFG,Afghanistan,1511.0,638.0,1154.0,492.0,430.0,2787.0,3013.0,1989.0,...,1170.0,4561.0,10357.0,8107.0,14457.0,16199.0,18808.0,20320.0,31107.0,32455.0
1,ALB,Albania,12.0,17.0,NaN,NaN,0.0,9.0,28.0,10.0,...,136034.0,0.0,0.0,0.0,0.0,0.0,17.0,3.0,NaN,NaN
2,DZA,Algeria,112.0,41.0,63.0,0.0,25.0,18.0,112.0,103.0,...,4169.0,2634.0,2500.0,3975.0,20114.0,22553.0,22126.0,29584.0,20849.0,15527.0
3,AND,Andorra,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,29.0,53.0,119.0,11699.0,8523.0,4458.0,1449.0,1190.0,...,19820.0,21009.0,13368.0,15580.0,22822.0,22685.0,22589.0,30067.0,19714.0,29656.0


In order to convert all the numbers to integers or if NaN, then a string 'noData':

Get a list of all the column headers, excluding first two. 'code' and 'name'.

In [6]:
columnHeaders = list(casesDF)[2:]

In [7]:
for column in columnHeaders:
    casesDF[column] = casesDF[column].apply(lambda x: int(x) if x == x else 'noData')

In [8]:
casesDF.head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,AFG,Afghanistan,1511,638,1154,492,430,2787,3013,1989,...,1170,4561,10357,8107,14457,16199,18808,20320,31107,32455
1,ALB,Albania,12,17,noData,noData,0,9,28,10,...,136034,0,0,0,0,0,17,3,noData,noData
2,DZA,Algeria,112,41,63,0,25,18,112,103,...,4169,2634,2500,3975,20114,22553,22126,29584,20849,15527
3,AND,Andorra,0,0,noData,noData,0,0,0,0,...,noData,noData,noData,noData,noData,noData,noData,noData,noData,noData
4,AGO,Angola,29,53,119,11699,8523,4458,1449,1190,...,19820,21009,13368,15580,22822,22685,22589,30067,19714,29656


## Merge in the 2018 data

Load in the CSV file from `measles-2018-case-data`

In [16]:
cases2018_2019DF = pd.read_csv(
    '../raw_data/cases/cases-2018-2019.csv',
)
cases2018_2019DF.head()

,code,2019,2018
0,DZA,NaN,287.0
1,AGO,2864.0,57.0
2,BEN,508.0,886.0
3,BWA,0.0,2.0
4,BFA,689.0,1682.0


Convert all the numbers to integers or if NaN, then a string 'noData':

In [17]:
cases2018_2019DF["2018"] = cases2018_2019DF["2018"].apply(lambda x: int(x) if x == x else 'noData')

In [18]:
cases2018_2019DF["2019"] = cases2018_2019DF["2019"].apply(lambda x: int(x) if x == x else 'noData')

In [19]:
cases2018_2019DF.head()

,code,2019,2018
0,DZA,noData,287
1,AGO,2864,57
2,BEN,508,886
3,BWA,0,2
4,BFA,689,1682


In [20]:
cases2018_2019DF[cases2018_2019DF['code'] == 'USA']

,code,2019,2018
79,USA,1095,372


In [21]:
mergedCasesDF = pd.merge(casesDF, cases2018_2019DF, on='code', how='left')

In [22]:
mergedCasesDF.head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1987,1986,1985,1984,1983,1982,1981,1980,2019,2018
0,AFG,Afghanistan,1511,638,1154,492,430,2787,3013,1989,...,10357,8107,14457,16199,18808,20320,31107,32455,106,1899
1,ALB,Albania,12,17,noData,noData,0,9,28,10,...,0,0,0,0,17,3,noData,noData,459,1466
2,DZA,Algeria,112,41,63,0,25,18,112,103,...,2500,3975,20114,22553,22126,29584,20849,15527,noData,287
3,AND,Andorra,0,0,noData,noData,0,0,0,0,...,noData,noData,noData,noData,noData,noData,noData,noData,0,0
4,AGO,Angola,29,53,119,11699,8523,4458,1449,1190,...,13368,15580,22822,22685,22589,30067,19714,29656,2864,57


Reorder the columns for convenience

In [23]:
sortedMergedCasesDF = mergedCasesDF.reindex(columns = sorted(mergedCasesDF.columns))

In [24]:
sortedMergedCasesDF.head()

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2012,2013,2014,2015,2016,2017,2018,2019,code,name
0,32455,31107,20320,18808,16199,14457,8107,10357,4561,1170,...,2787,430,492,1154,638,1511,1899,106,AFG,Afghanistan
1,noData,noData,3,17,0,0,0,0,0,136034,...,9,0,noData,noData,17,12,1466,459,ALB,Albania
2,15527,20849,29584,22126,22553,20114,3975,2500,2634,4169,...,18,25,0,63,41,112,287,noData,DZA,Algeria
3,noData,noData,noData,noData,noData,noData,noData,noData,noData,noData,...,0,0,noData,noData,0,0,0,0,AND,Andorra
4,29656,19714,30067,22589,22685,22822,15580,13368,21009,19820,...,4458,8523,11699,119,53,29,57,2864,AGO,Angola


## Load in the vaccination data

From http://apps.who.int/gho/data/view.main.80100?lang=en

In [27]:
!curl 'http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHS8_110&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR' -o '../raw_data/vaccination/first-dose/xmart.csv'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41963    0 41963    0     0   267k      0 --:--:-- --:--:-- --:--:--  267k


In [29]:
vacDF = pd.read_csv(
    '../raw_data/vaccination/first-dose/xmart.csv',
    skiprows=1)

In [30]:
vacDF.head()

,Country,2017,2016,2015,2014,2013,2012,2011,2010,2009,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,Afghanistan,62,62,62,60,57,59,64,62,60,...,22,34,31,14,14,14,9,8,NaN,11
1,Albania,96,96,97,98,99,98,99,99,97,...,96,96,96,96,96,96,96,93,90,90
2,Algeria,88,94,95,95,95,95,95,95,92,...,82,81,73,67,68,NaN,NaN,NaN,NaN,NaN
3,Andorra,99,97,96,96,95,98,99,99,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,42,45,51,56,59,67,60,67,53,...,48,56,55,44,44,35,26,NaN,NaN,NaN


There is a space before each column name...

In [31]:
vacDF[" 2017"].head()

0    62
1    96
2    88
3    99
4    42
Name:  2017, dtype: int64

In [32]:
vacDF.columns = vacDF.columns.str.lstrip()

This is now removed

In [33]:
vacDF["2017"].head()

0    62
1    96
2    88
3    99
4    42
Name: 2017, dtype: int64

Rename 'Country' to 'name'

In [34]:
vacDF.rename(index=str, columns={"Country":"name"}, inplace=True)
vacDF.head()

,name,2017,2016,2015,2014,2013,2012,2011,2010,2009,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,Afghanistan,62,62,62,60,57,59,64,62,60,...,22,34,31,14,14,14,9,8,NaN,11
1,Albania,96,96,97,98,99,98,99,99,97,...,96,96,96,96,96,96,96,93,90,90
2,Algeria,88,94,95,95,95,95,95,95,92,...,82,81,73,67,68,NaN,NaN,NaN,NaN,NaN
3,Andorra,99,97,96,96,95,98,99,99,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,42,45,51,56,59,67,60,67,53,...,48,56,55,44,44,35,26,NaN,NaN,NaN


## Add ISO_codes to the vaccination data

Use `CountryInfo` to get the iso code from a country name

In [35]:
try:
    print(CountryInfo('Venezuela').iso(3))
except:
    print('There was an error')

VEN


Does not work with the full name with brackets

In [36]:
try:
    print(CountryInfo('Venezuela (Bolivarian Republic of)').iso(3))
except:
    print('There was an error')

There was an error


Use a regex to remove the brackets and space

In [37]:
re.sub('\s+\(.*?\)', '', 'Venezuela (Bolivarian Republic of)')

'Venezuela'

In [38]:
try:
    print(CountryInfo(re.sub('\s+\(.*?\)', '', 'Venezuela (Bolivarian Republic of)')).iso(3))
except:
    print('There was an error')

VEN


Create a dictionary of the names that CountryInfo can't handle...

In [39]:
nameExceptions = {"Andorra":"AND",
"Bahamas":"BHS",
"Brunei Darussalam":"BRN",
"Cabo Verde":"CPV",
"Congo":"COG",
"CíÇte d'Ivoire":"CIV",
"Côte d'Ivoire": "CIV",
"Czechia":"CZE",
"Democratic People's Republic of Korea":"PRK",
"Eswatini":"SWZ",
"Gambia":"GMB",
"Lao People's Democratic Republic":"LAO",
"Micronesia (Federated States of)":"FSM",
"Montenegro":"MNE",
"Myanmar":"MMR",
"Republic of Korea":"KOR",
"Republic of Moldova":"MDA",
"Republic of North Macedonia": "MKD",
"Russian Federation":"RUS",
"Sao Tome and Principe":"STP",
"Serbia":"SRB",
"Syrian Arab Republic":"SYR",
"The former Yugoslav republic of Macedonia":"MKD",
"Timor-Leste":"TLS",
"United Kingdom of Great Britain and Northern Ireland":"GBR",
"United Republic of Tanzania":"TZA",
"United States of America":"USA",
"Viet Nam":"VNM"}

In [40]:
nameExceptions["CíÇte d'Ivoire"]

'CIV'

Loop through the country names and apply insert an iso code into a new list

In [41]:
codes = []
for name in vacDF['name']:
    newName = re.sub('\s+\(.*?\)', '', name)
    code = ''
    try:
        code = CountryInfo(newName).iso(3)
    except:
        code = nameExceptions[name]
    codes.append(code)

In [42]:
codes[0:10]

['AFG', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM', 'AUS', 'AUT']

In [43]:
vacDF.insert(loc=0, column="code", value=codes)
vacDF.head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,AFG,Afghanistan,62,62,62,60,57,59,64,62,...,22,34,31,14,14,14,9,8,NaN,11
1,ALB,Albania,96,96,97,98,99,98,99,99,...,96,96,96,96,96,96,96,93,90,90
2,DZA,Algeria,88,94,95,95,95,95,95,95,...,82,81,73,67,68,NaN,NaN,NaN,NaN,NaN
3,AND,Andorra,99,97,96,96,95,98,99,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,42,45,51,56,59,67,60,67,...,48,56,55,44,44,35,26,NaN,NaN,NaN


## Something funny is happening with the Syrian data

In [44]:
vacDF.loc[vacDF['code'] == 'SYR'].head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
168,SYR,Syrian Arab Republic,67,62 62,53 53,54 54,58 58,61 61,80 80,82 82,...,86 86,52 52,37 37,64 64,27 27,23 23,19 19,22 22,14 14,13 13


In [45]:
vacColumnHeaders = vacDF.columns[2:]

In [46]:
def testEntry(x):
    try:
        int(x)
        return int(x)
    except:
        return 'noData'

In [47]:
testEntry('15 15')

'noData'

In [48]:
for column in vacColumnHeaders:
    vacDF[column] = vacDF[column].apply(testEntry)

In [49]:
vacDF.loc[vacDF['code'] == 'SYR'].head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
168,SYR,Syrian Arab Republic,67,noData,noData,noData,noData,noData,noData,noData,...,noData,noData,noData,noData,noData,noData,noData,noData,noData,noData


## Add a placeholder column for 2018 and 2019

In [50]:
vacDF['2018'] = 'noData'
vacDF['2019'] = 'noData'

In [51]:
vacDF.head()

,code,name,2017,2016,2015,2014,2013,2012,2011,2010,...,1987,1986,1985,1984,1983,1982,1981,1980,2018,2019
0,AFG,Afghanistan,62,62,62,60,57,59,64,62,...,31,14,14,14,9,8,noData,11,noData,noData
1,ALB,Albania,96,96,97,98,99,98,99,99,...,96,96,96,96,96,93,90,90,noData,noData
2,DZA,Algeria,88,94,95,95,95,95,95,95,...,73,67,68,noData,noData,noData,noData,noData,noData,noData
3,AND,Andorra,99,97,96,96,95,98,99,99,...,noData,noData,noData,noData,noData,noData,noData,noData,noData,noData
4,AGO,Angola,42,45,51,56,59,67,60,67,...,55,44,44,35,26,noData,noData,noData,noData,noData


## Sort the columns just so they match the cases data

In [52]:
sortedVacDF = vacDF.reindex(columns = sorted(vacDF.columns))
sortedVacDF.head()

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2012,2013,2014,2015,2016,2017,2018,2019,code,name
0,11,noData,8,9,14,14,14,31,34,22,...,59,57,60,62,62,62,noData,noData,AFG,Afghanistan
1,90,90,93,96,96,96,96,96,96,96,...,98,99,98,97,96,96,noData,noData,ALB,Albania
2,noData,noData,noData,noData,noData,68,67,73,81,82,...,95,95,95,95,94,88,noData,noData,DZA,Algeria
3,noData,noData,noData,noData,noData,noData,noData,noData,noData,noData,...,98,95,96,96,97,99,noData,noData,AND,Andorra
4,noData,noData,noData,26,35,44,44,55,56,48,...,67,59,56,51,45,42,noData,noData,AGO,Angola


## Print the dataframes to CSVs

In [56]:
sortedMergedCasesDF.to_csv("../raw_data/cases/cases-by-who-region-2019.csv", index=False, encoding="utf-8", na_rep="noData")

In [57]:
sortedVacDF.to_csv("../raw_data/vaccination/vaccination-by-who-region-2019.csv", index=False, encoding="utf-8", na_rep="noData")